ECO 7110 – Econometrics II

Abrar Bagalb, Robert Schultz, and Ryan Thomas

OCTOBER 2, 2019

            FE,RE,and CRE Lab

(a)Estimate the model by RE, and interpret the coefficient on the in-season 
variable


In [0]:
!pip install linearmodels
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from statsmodels import regression
from statsmodels.stats import diagnostic
import statsmodels.regression as smr
import statsmodels.api as sm
import linearmodels as lm
from linearmodels.iv import IV2SLS
import scipy as sp
import scipy.stats
file="https://github.com/VitorKamada/ECO7110/raw/master/Data/gpa.dta"
data = pd.read_stata(file)
term8808 = pd.Categorical(data.term)
idt = pd.Categorical(data['id'])
data['term8808']= term8808
data['idt']= idt
data = data.set_index(['idt', 'term'])
exog_vars = ['spring','crsgpa','frstsem','season',
           'sat', 'verbmath', 'hsperc', 'hssize', 'black', 'female']
exog = sm.add_constant(data[exog_vars])
OLS = PooledOLS(data.trmgpa, 
 exog).fit(cov_type='clustered', cluster_entity=True)
from linearmodels.panel import RandomEffects
RE = RandomEffects(data.trmgpa, 
 exog).fit(cov_type='clustered', cluster_entity=True)
print(RE)


                        RandomEffects Estimation Summary                        
Dep. Variable:                 trmgpa   R-squared:                        0.4167
Estimator:              RandomEffects   R-squared (Between):              0.5389
No. Observations:                 732   R-squared (Within):               0.2061
Date:                Wed, Oct 02 2019   R-squared (Overall):              0.4785
Time:                        19:00:00   Log-likelihood                   -380.32
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      51.505
Entities:                         366   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                  F(10,721)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             58.564
                            

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


The coefficient on season is -0.04 which means that being in season is estimated to reduce term GPA by 0.04 points.

b)Estimate the model by FE;informally compare the estimates to the RE estimates,
in particular that on the in-season effect.

In [0]:
from linearmodels.panel import PanelOLS
FE = PanelOLS(data.trmgpa, exog, entity_effects=True, 
 drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True)
print(FE)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 trmgpa   R-squared:                        0.2069
Estimator:                   PanelOLS   R-squared (Between):              0.0213
No. Observations:                 732   R-squared (Within):               0.2069
Date:                Wed, Oct 02 2019   R-squared (Overall):              0.0550
Time:                        17:52:07   Log-likelihood                   -126.20
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      23.611
Entities:                         366   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                   F(4,362)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             19.555
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/model.py:1316: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

sat, verbmath, hsperc, hssize, black, female

  AbsorbingEffectWarning)


 The coefficient on the season is larger in FE (-0.056), and it is more statistically significant

c)Construct the nonrobust Hausman test comparing RE and FE. Include all 
variables in W that have some varation across i and t, except for the term dummy

In [0]:
from linearmodels.panel import compare
print(compare({'OLS':OLS,'RE': RE, 'FE': FE}))
data['crsgpabar'] = data.groupby(idt)['crsgpa'].transform('mean')
data['frstsembar'] = data.groupby(idt)['frstsem'].transform('mean')
data['seasonbar'] = data.groupby(idt)['season'].transform('mean')
Exog = ['spring','crsgpa','frstsem','season',
           'sat', 'verbmath', 'hsperc', 'hssize', 'black', 'female', 'crsgpabar', 'frstsembar', 'seasonbar']

Exog = sm.add_constant(data[Exog])
CRE = RandomEffects(data.trmgpa,Exog).fit(cov_type='clustered', cluster_entity=True)
print(CRE)


                           Model Comparison                           
                                   FE            OLS                RE
----------------------------------------------------------------------
Dep. Variable                  trmgpa         trmgpa            trmgpa
Estimator                    PanelOLS      PooledOLS     RandomEffects
No. Observations                  732            732               732
Cov. Est.                   Clustered      Clustered         Clustered
R-squared                      0.2069         0.4787            0.4167
R-Squared (Within)             0.2069         0.2041            0.2061
R-Squared (Between)            0.0213         0.5396            0.5389
R-Squared (Overall)            0.0550         0.4787            0.4785
F-statistic                    23.611         66.215            51.505
P-value (F-stat)               0.0000         0.0000            0.0000
=====================     ===========   ============   ===============
const 

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                        RandomEffects Estimation Summary                        
Dep. Variable:                 trmgpa   R-squared:                        0.4191
Estimator:              RandomEffects   R-squared (Between):              0.5408
No. Observations:                 732   R-squared (Within):               0.2069
Date:                Wed, Oct 02 2019   R-squared (Overall):              0.4802
Time:                        17:50:11   Log-likelihood                   -381.57
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      39.848
Entities:                         366   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                  F(13,718)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             48.508
                            